# Normalize Form Fields Classifier

Thoughts:
- **Make this work with new package**
- fill training data with forms selected based on connections based on random sampeling in jur
- make the training only take palce if source and labeled are different (filter on old_fields = new fields)
- don't lable auto-fields label with adjacent text from another field use `_ignore_`
- consider training only within jurisdiction
- should I norm vectors in reormating function?

## Load Libraries

In [1]:
jurs = ["MI", "UT", "ME"]

In [17]:
import pandas as pd
import re
from os import walk
from sklearn.metrics import accuracy_score

if 1 == 2:
    import PyPDF2
    from sklearn.preprocessing import normalize
    import urllib.request, json
    from urllib.request import urlopen
    from bs4 import BeautifulSoup
    import os.path
    from os import path
    import numpy as np

    #!pip install py-readability-metrics
    from readability import Readability

    #!python -m nltk.downloader punkt

    import networkx as nx

    from matplotlib.pyplot import figure

    from pyvis.network import Network

    import time
    from datetime import date

    import spacy

    from sklearn.metrics import accuracy_score

    from sklearn.model_selection import KFold

    from sklearn.linear_model import LogisticRegression
    from sklearn import tree
    from sklearn.ensemble import (
        RandomForestClassifier,
        AdaBoostClassifier,
        VotingClassifier,
    )
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import GradientBoostingClassifier

    from joblib import dump, load

    import math

    from sklearn.metrics.pairwise import (
        cosine_similarity,
    )  # define matrix with all zero values

    # import nltk
    # nltk.download('stopwords')

    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize

In [3]:
import sys

sys.path.insert(0, "../../formfyxer")
import formfyxer.lit_explorer as ff

## Get training and test form data from each of our jurisdictions

First we'll start by grabing a set of forms with automaticly-generated form field names.

In [4]:
included_fields = [
    "users1_name",
    "users1_birthdate",
    "users1_address_line_one",
    "users1_address_line_two",
    "users1_address_city",
    "users1_address_state",
    "users1_address_zip",
    "users1_phone_number",
    "users1_email",
    "plantiffs1_name",
    "defendants1_name",
    "petitioners1_name",
    "respondents1_name",
    "docket_number",
    "trial_court_county",
    "users1_signature",
    "signature_date",
]

In [7]:
path = "ML_training/auto/"
f = []
for dirpath, dirnames, filenames in walk(path):
    f.extend(filenames)
    break
f1 = [file for file in f]

auto_df = pd.DataFrame(
    [], columns=["form_id", "jurisdiction", "group", "n", "per", "this_field"]
)

for file in f1:
    if re.search(".*\.pdf", file):
        form_id = re.search("(.*)\.pdf", file).groups()[0]
        print(path + file)
        stats = ff.parse_form(path + file, "title", "group")
        i = 0
        n_fields = len(stats["fields"])
        last_field = "null"
        for stat in stats["fields"]:
            # print(form_id,df[df["id"]==form_id]["jurisdiction"].values[0])
            # auto_df = auto_df.append(pd.DataFrame([[form_id,df[df["id"]==form_id]["jurisdiction"].values[0],df[df["id"]==form_id]["group"].values[0],i,(i/n_fields),lit.reCase(stat)]],columns=["form_id","jurisdiction","group","n","per","this_field"]), ignore_index=True,sort=False)
            auto_df = auto_df.append(
                pd.DataFrame(
                    [[form_id, "state", "group", i, (i / n_fields), ff.reCase(stat)]],
                    columns=[
                        "form_id",
                        "jurisdiction",
                        "group",
                        "n",
                        "per",
                        "this_field",
                    ],
                ),
                ignore_index=True,
                sort=False,
            )
            i += 1

auto_df

ML_training/auto/9dda7eb4b8b64f88abd092075ce6245d.pdf
ML_training/auto/9ec36f386c34421b9da9f9ce6082a5ce.pdf
ML_training/auto/3a09aadafc4a4732b41ad48fd313ffe8.pdf
ML_training/auto/095b9dc651ce47eb8b62e0790974970f.pdf
ML_training/auto/b8dc2bf7b60946f2bbbf45cc3967f78f.pdf
ML_training/auto/f7e6e67e408c4309b83b8a6fde22b706.pdf
ML_training/auto/3902bb0b832b4fa4b20e7635201017aa.pdf
ML_training/auto/e5e3527acd804e7581f86e14fd41f86c.pdf
ML_training/auto/ebb2721315f845819b46cf395f956966.pdf
ML_training/auto/d488e5e1f3d44d3a99c50a4ec5305af8.pdf


,form_id,jurisdiction,group,n,per,this_field
0,9dda7eb4b8b64f88abd092075ce6245d,state,group,0,0.000000,plaintiffs address
1,9dda7eb4b8b64f88abd092075ce6245d,state,group,1,0.050000,patty
2,9dda7eb4b8b64f88abd092075ce6245d,state,group,2,0.100000,defendants address
3,9dda7eb4b8b64f88abd092075ce6245d,state,group,3,0.150000,datty
4,9dda7eb4b8b64f88abd092075ce6245d,state,group,4,0.200000,date hearing
...,...,...,...,...,...,...
314,d488e5e1f3d44d3a99c50a4ec5305af8,state,group,17,0.772727,*users 1 signature _2
315,d488e5e1f3d44d3a99c50a4ec5305af8,state,group,18,0.818182,*users 1 name _2
316,d488e5e1f3d44d3a99c50a4ec5305af8,state,group,19,0.863636,exf approved march
317,d488e5e1f3d44d3a99c50a4ec5305af8,state,group,20,0.909091,order petition expunge records cannabis convic...


In [8]:
auto_df["this_field"].unique()

array(['plaintiffs address', 'patty', 'defendants address', 'datty',
       'date hearing', 'judge', 'unknown _1', 'circuit', 'county',
       '*docket number', '*users 1 address line one', 'telno',
       'plaintiffs name', 'defendants name', '13 a', 'unknown _2', 'b',
       '*signature date _1', '*signature date _2', 'bar', 'cirno',
       'casename', 'lhofcr', 'chd', 'individual', '*users 1 address city',
       '*users 1 address state', '*users 1 address zip', 'location',
       'item _1', 'odate', 'item _2', 'ndate', 'disease', 'item _3',
       'item b_ 1', 'item c_ 1', 'item _4', 'explain _1', 'explain _2',
       'item _5', 'explain _3', 'item b_ 2', 'explain _4', 'item _6',
       'explain _5', 'explain _6', 'item _7', 'edu', 'treat', 'couns',
       'item b_ 3', 'item c_ 2', 'item _8', 'place', 'item e', 'item f',
       'other 9', 'item _9', 'facility _1', 'pdate', 'aname', 'aaddress',
       'acity', 'atelno', 'ptelno', 'pname', 'paddress', 'pcity',
       'text _1', 'text

Now we'll grab the same set of forms with ground truth label names. 

In [9]:
path = "ML_training/ground/"
f = []
for dirpath, dirnames, filenames in walk(path):
    f.extend(filenames)
    break
f1 = [file for file in f]

ground_df = pd.DataFrame(
    [], columns=["form_id", "n"] + included_fields + ["this_field_ground"]
)

for file in f1:
    if re.search(".*\.pdf", file):
        form_id = re.search("(.*)\.pdf", file).groups()[0]
        print(path + file)
        stats = ff.parse_form(path + file, "title", "group")
        i = 0
        n_fields = len(stats["fields"])
        last_field = "null"
        for stat in stats["fields"]:
            # print("\t"+str(i)+","+str(i/n_fields)+","+last_field+","+stat[:5])
            if "_ignore_" not in stat:
                #
                # this * is a hold over from form explorer that should be removed
                if re.sub("^\*", "", stat) not in included_fields:
                    stat = "_nonstandard_"
                field_matches = []
                for item in included_fields:
                    if last_field == item:
                        field_matches.append(1)
                    else:
                        field_matches.append(0)
                ground_df = ground_df.append(
                    pd.DataFrame(
                        [[form_id, i] + field_matches + [stat]],
                        columns=["form_id", "n"]
                        + included_fields
                        + ["this_field_ground"],
                    ),
                    ignore_index=True,
                    sort=False,
                )
            last_field = stat
            i += 1

ground_df

ML_training/ground/9dda7eb4b8b64f88abd092075ce6245d.pdf
ML_training/ground/9ec36f386c34421b9da9f9ce6082a5ce.pdf
ML_training/ground/3a09aadafc4a4732b41ad48fd313ffe8.pdf
ML_training/ground/095b9dc651ce47eb8b62e0790974970f.pdf
ML_training/ground/b8dc2bf7b60946f2bbbf45cc3967f78f.pdf
ML_training/ground/f7e6e67e408c4309b83b8a6fde22b706.pdf
ML_training/ground/3902bb0b832b4fa4b20e7635201017aa.pdf
ML_training/ground/e5e3527acd804e7581f86e14fd41f86c.pdf
ML_training/ground/ebb2721315f845819b46cf395f956966.pdf
ML_training/ground/d488e5e1f3d44d3a99c50a4ec5305af8.pdf


,form_id,n,users1_name,users1_birthdate,users1_address_line_one,users1_address_line_two,users1_address_city,users1_address_state,users1_address_zip,users1_phone_number,users1_email,plantiffs1_name,defendants1_name,petitioners1_name,respondents1_name,docket_number,trial_court_county,users1_signature,signature_date,this_field_ground
0,9dda7eb4b8b64f88abd092075ce6245d,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,_nonstandard_
1,9dda7eb4b8b64f88abd092075ce6245d,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,_nonstandard_
2,9dda7eb4b8b64f88abd092075ce6245d,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,_nonstandard_
3,9dda7eb4b8b64f88abd092075ce6245d,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,_nonstandard_
4,9dda7eb4b8b64f88abd092075ce6245d,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,_nonstandard_
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,d488e5e1f3d44d3a99c50a4ec5305af8,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,*users1_name
302,d488e5e1f3d44d3a99c50a4ec5305af8,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,*users1_address_line_one
303,d488e5e1f3d44d3a99c50a4ec5305af8,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,*users1_address_line_two
304,d488e5e1f3d44d3a99c50a4ec5305af8,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,*users1_email


In [10]:
ground_df["this_field_ground"].unique()

array(['_nonstandard_', '*users1_address_line_one', '*trial_court_county',
       '*docket_number', '*signature_date', '*users1_address_city',
       '*users1_address_state', '*users1_address_zip', '*users1_name',
       '*users1_birthdate', '*users1_address_line_two', '*users1_email',
       '*users1_phone_number', '*users1_signature', '*respondents1_name',
       '*petitioners1_name'], dtype=object)

## All together now

Let's merge the form data and create some features. 

In [11]:
df_all = auto_df.merge(ground_df, how="inner")
print(len(df_all))
display(df_all.head())

# take care of some categorical variables
parts = pd.get_dummies(df_all["jurisdiction"])
parts = pd.concat([parts, pd.get_dummies(df_all["group"])], axis=1)
df_all = pd.concat([df_all, parts], axis=1)
jurisdictions = df_all["jurisdiction"].unique().tolist()

# groups = df_all['group'].unique().tolist()
groups = ["state"]

df_all = df_all.drop(["jurisdiction", "group"], 1, errors="ignore").copy()
display(df_all.head())
df_all["this_field_vec"] = df_all["this_field"].apply(ff.vectorize)
# df_all["last_field_ground_vec"] = df_all["last_field_ground"].apply(vectorize)
df_all = df_all.drop(["this_field", "last_field_ground"], 1, errors="ignore").copy()
display(df_all.head())
# df_all["last_field_ground_vec"] = df_all["last_field_ground_vec"].apply(norm)
display(df_all.head())
df_all = pd.concat(
    [
        df_all,
        pd.DataFrame(
            df_all.this_field_vec.values.tolist(), index=df_all.index
        ).add_prefix("vec_this"),
    ],
    axis=1,
)
# df_all = pd.concat([df_all, pd.DataFrame(df_all.last_field_ground_vec.values.tolist(), index= df_all.index).add_prefix("vec_last")], axis=1)
# df_all = df_all.drop(['this_field_vec','last_field_ground_vec'], 1, errors='ignore').copy()
df_all = df_all.drop(["this_field_vec"], 1, errors="ignore").copy()
display(df_all.head())

305


,form_id,jurisdiction,group,n,per,this_field,users1_name,users1_birthdate,users1_address_line_one,users1_address_line_two,...,users1_email,plantiffs1_name,defendants1_name,petitioners1_name,respondents1_name,docket_number,trial_court_county,users1_signature,signature_date,this_field_ground
0,9dda7eb4b8b64f88abd092075ce6245d,state,group,0,0.00,plaintiffs address,0,0,0,0,...,0,0,0,0,0,0,0,0,0,_nonstandard_
1,9dda7eb4b8b64f88abd092075ce6245d,state,group,1,0.05,patty,0,0,0,0,...,0,0,0,0,0,0,0,0,0,_nonstandard_
2,9dda7eb4b8b64f88abd092075ce6245d,state,group,2,0.10,defendants address,0,0,0,0,...,0,0,0,0,0,0,0,0,0,_nonstandard_
3,9dda7eb4b8b64f88abd092075ce6245d,state,group,3,0.15,datty,0,0,0,0,...,0,0,0,0,0,0,0,0,0,_nonstandard_
4,9dda7eb4b8b64f88abd092075ce6245d,state,group,4,0.20,date hearing,0,0,0,0,...,0,0,0,0,0,0,0,0,0,_nonstandard_


,form_id,n,per,this_field,users1_name,users1_birthdate,users1_address_line_one,users1_address_line_two,users1_address_city,users1_address_state,...,plantiffs1_name,defendants1_name,petitioners1_name,respondents1_name,docket_number,trial_court_county,users1_signature,signature_date,this_field_ground,state
0,9dda7eb4b8b64f88abd092075ce6245d,0,0.00,plaintiffs address,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,_nonstandard_,1
1,9dda7eb4b8b64f88abd092075ce6245d,1,0.05,patty,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,_nonstandard_,1
2,9dda7eb4b8b64f88abd092075ce6245d,2,0.10,defendants address,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,_nonstandard_,1
3,9dda7eb4b8b64f88abd092075ce6245d,3,0.15,datty,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,_nonstandard_,1
4,9dda7eb4b8b64f88abd092075ce6245d,4,0.20,date hearing,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,_nonstandard_,1


,form_id,n,per,users1_name,users1_birthdate,users1_address_line_one,users1_address_line_two,users1_address_city,users1_address_state,users1_address_zip,...,defendants1_name,petitioners1_name,respondents1_name,docket_number,trial_court_county,users1_signature,signature_date,this_field_ground,state,this_field_vec
0,9dda7eb4b8b64f88abd092075ce6245d,0,0.00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[0.0037960527438874953, 0.004272600604347598, ..."
1,9dda7eb4b8b64f88abd092075ce6245d,1,0.05,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[-0.001815179919829267, 0.0022052016057158485,..."
2,9dda7eb4b8b64f88abd092075ce6245d,2,0.10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[0.002004389025620966, 0.0038552705780302973, ..."
3,9dda7eb4b8b64f88abd092075ce6245d,3,0.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,9dda7eb4b8b64f88abd092075ce6245d,4,0.20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[0.0014712272439873927, 0.0033801516020308983,..."


,form_id,n,per,users1_name,users1_birthdate,users1_address_line_one,users1_address_line_two,users1_address_city,users1_address_state,users1_address_zip,...,defendants1_name,petitioners1_name,respondents1_name,docket_number,trial_court_county,users1_signature,signature_date,this_field_ground,state,this_field_vec
0,9dda7eb4b8b64f88abd092075ce6245d,0,0.00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[0.0037960527438874953, 0.004272600604347598, ..."
1,9dda7eb4b8b64f88abd092075ce6245d,1,0.05,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[-0.001815179919829267, 0.0022052016057158485,..."
2,9dda7eb4b8b64f88abd092075ce6245d,2,0.10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[0.002004389025620966, 0.0038552705780302973, ..."
3,9dda7eb4b8b64f88abd092075ce6245d,3,0.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,9dda7eb4b8b64f88abd092075ce6245d,4,0.20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[0.0014712272439873927, 0.0033801516020308983,..."


,form_id,n,per,users1_name,users1_birthdate,users1_address_line_one,users1_address_line_two,users1_address_city,users1_address_state,users1_address_zip,...,vec_this290,vec_this291,vec_this292,vec_this293,vec_this294,vec_this295,vec_this296,vec_this297,vec_this298,vec_this299
0,9dda7eb4b8b64f88abd092075ce6245d,0,0.00,0,0,0,0,0,0,0,...,-0.000228,0.000896,0.000741,0.003968,0.000544,-0.007890,0.002105,-0.004488,-0.002620,0.002919
1,9dda7eb4b8b64f88abd092075ce6245d,1,0.05,0,0,0,0,0,0,0,...,0.013065,0.002901,-0.002514,-0.004821,-0.003409,-0.003333,-0.001171,-0.010150,0.001421,-0.000092
2,9dda7eb4b8b64f88abd092075ce6245d,2,0.10,0,0,0,0,0,0,0,...,-0.004533,0.002606,-0.000724,0.005710,-0.001702,-0.009501,0.004274,-0.004080,-0.003250,0.004680
3,9dda7eb4b8b64f88abd092075ce6245d,3,0.15,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,9dda7eb4b8b64f88abd092075ce6245d,4,0.20,0,0,0,0,0,0,0,...,-0.000558,0.001629,0.000205,-0.000542,0.001426,-0.004128,-0.007151,0.002678,-0.000516,0.000274


In [12]:
df_class = pd.concat([df_all[["form_id"]], df_all[jurisdictions]], axis=1)
df_class = pd.concat([df_class, df_all[["n", "per"]]], axis=1)
df_class = pd.concat([df_class, df_all[groups], df_all[included_fields]], axis=1)
df_class = pd.concat(
    [
        df_class,
        df_all[[x for x in df_all.columns if "vec_" in x]],
        df_all[["this_field_ground"]],
    ],
    axis=1,
)
df_class

,form_id,state,n,per,state,users1_name,users1_birthdate,users1_address_line_one,users1_address_line_two,users1_address_city,...,vec_this291,vec_this292,vec_this293,vec_this294,vec_this295,vec_this296,vec_this297,vec_this298,vec_this299,this_field_ground
0,9dda7eb4b8b64f88abd092075ce6245d,1,0,0.000000,1,0,0,0,0,0,...,0.000896,0.000741,0.003968,0.000544,-0.007890,0.002105,-0.004488,-0.002620,0.002919,_nonstandard_
1,9dda7eb4b8b64f88abd092075ce6245d,1,1,0.050000,1,0,0,0,0,0,...,0.002901,-0.002514,-0.004821,-0.003409,-0.003333,-0.001171,-0.010150,0.001421,-0.000092,_nonstandard_
2,9dda7eb4b8b64f88abd092075ce6245d,1,2,0.100000,1,0,0,0,0,0,...,0.002606,-0.000724,0.005710,-0.001702,-0.009501,0.004274,-0.004080,-0.003250,0.004680,_nonstandard_
3,9dda7eb4b8b64f88abd092075ce6245d,1,3,0.150000,1,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,_nonstandard_
4,9dda7eb4b8b64f88abd092075ce6245d,1,4,0.200000,1,0,0,0,0,0,...,0.001629,0.000205,-0.000542,0.001426,-0.004128,-0.007151,0.002678,-0.000516,0.000274,_nonstandard_
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,d488e5e1f3d44d3a99c50a4ec5305af8,1,15,0.681818,1,0,0,0,0,0,...,-0.000608,0.001173,0.003171,0.004593,-0.004891,-0.001934,-0.001439,-0.004086,0.000356,*users1_name
301,d488e5e1f3d44d3a99c50a4ec5305af8,1,16,0.727273,1,0,0,0,0,0,...,0.006506,-0.003021,-0.003436,-0.000011,-0.004560,-0.002875,-0.001439,0.001978,-0.000334,*users1_address_line_one
302,d488e5e1f3d44d3a99c50a4ec5305af8,1,17,0.772727,1,0,0,0,0,0,...,0.005727,-0.001891,-0.002049,-0.003148,-0.002341,-0.001658,-0.005468,0.001658,0.004401,*users1_address_line_two
303,d488e5e1f3d44d3a99c50a4ec5305af8,1,18,0.818182,1,0,0,0,0,0,...,0.002876,-0.002977,-0.000035,-0.003397,-0.002669,-0.004311,-0.006235,0.002061,0.004173,*users1_email


In [13]:
# down sample
if 1 == 1:
    df0 = df_class[df_class["this_field_ground"] != "_nonstandard_"]
    df1 = df_class[df_class["this_field_ground"] == "_nonstandard_"][
        : len(df_class[df_class["this_field_ground"] != "_nonstandard_"])
    ]
    df_class = pd.concat([df0, df1], ignore_index=True)

    display(df_class)

,form_id,state,n,per,state,users1_name,users1_birthdate,users1_address_line_one,users1_address_line_two,users1_address_city,...,vec_this291,vec_this292,vec_this293,vec_this294,vec_this295,vec_this296,vec_this297,vec_this298,vec_this299,this_field_ground
0,9dda7eb4b8b64f88abd092075ce6245d,1,8,0.400000,1,0,0,0,0,0,...,0.001980,0.002938,0.001264,-0.006187,-0.005235,0.005668,-0.007016,-0.004931,-0.005415,*users1_address_line_one
1,9dda7eb4b8b64f88abd092075ce6245d,1,16,0.800000,1,0,0,0,0,0,...,-0.007208,-0.007990,-0.001361,0.006704,0.000299,-0.002564,-0.002394,0.001156,0.001749,*trial_court_county
2,9dda7eb4b8b64f88abd092075ce6245d,1,17,0.850000,1,0,0,0,0,0,...,0.007806,-0.003096,-0.003656,-0.000196,-0.005329,-0.003647,-0.001955,0.001666,0.000348,*docket_number
3,9dda7eb4b8b64f88abd092075ce6245d,1,18,0.900000,1,0,0,0,0,0,...,0.006506,-0.003021,-0.003436,-0.000011,-0.004560,-0.002875,-0.001439,0.001978,-0.000334,*signature_date
4,9ec36f386c34421b9da9f9ce6082a5ce,1,6,0.103448,1,0,0,0,0,0,...,0.004354,-0.000257,-0.001725,-0.001159,0.000018,-0.002144,-0.006790,0.001848,0.000664,*users1_address_city
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,9ec36f386c34421b9da9f9ce6082a5ce,1,41,0.706897,1,0,0,0,0,0,...,0.002537,-0.001572,0.000341,0.000116,-0.006330,-0.007096,-0.004200,0.002081,0.000158,_nonstandard_
112,9ec36f386c34421b9da9f9ce6082a5ce,1,42,0.724138,1,0,0,0,0,0,...,0.001176,0.002430,-0.002310,-0.002903,-0.006038,-0.004528,-0.002443,-0.003196,0.003998,_nonstandard_
113,9ec36f386c34421b9da9f9ce6082a5ce,1,43,0.741379,1,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,_nonstandard_
114,9ec36f386c34421b9da9f9ce6082a5ce,1,44,0.758621,1,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,_nonstandard_


In [14]:
1 - len(df_class["this_field_ground"].unique()) / len(df_class["this_field_ground"])

0.8620689655172413

In [28]:
form_list = pd.DataFrame([], columns=["form_id"])
form_list["form_id"] = df_class["form_id"].unique()
holdout = form_list.sample(frac=0.2)

# create a dataframe containing a random sample of rows
class_holdout = df_class.loc[df_class.form_id.isin(holdout["form_id"])]

# create a dataframe that conatins the rows from except for those in holdout
class_training = df_class.loc[~df_all.form_id.isin(holdout["form_id"])]

# make a training dataframe containing just features
features_train = class_training.drop(["form_id", "this_field_ground"], axis=1).values

# make a training dataframe containing only the target
labels_train = class_training["this_field_ground"].values

# make a testing/holdout dataframe containing just features
features_test = class_holdout.drop(["form_id", "this_field_ground"], axis=1).values

# make a testing/holdout dataframe containing only the target
labels_test = class_holdout["this_field_ground"].values

print("size of training", len(class_training))
print("size of testing", len(class_holdout))

size of training 111
size of testing 10


In [29]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(fit_intercept=False, C=1e9)
clf_1 = model.fit(features_train, labels_train)
pred = clf_1.predict(features_test)
prob = clf_1.predict_proba(features_test)[:, 1]
print("Logistic Regression")

print(accuracy_score(labels_test, pred))
# evaluate(prob, pred, labels_test)

print("")
print(labels_test, "\n")
print(pred)

Logistic Regression
0.0

['*users1_address_line_one' '*docket_number' '*trial_court_county'
 '*signature_date' '*users1_name' '*users1_signature' '*signature_date'
 '*users1_name' '*users1_birthdate' '*trial_court_county'] 

['_nonstandard_' '*users1_address_line_one' '*users1_address_line_one'
 '*users1_address_line_one' '*users1_address_line_one' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '*docket_number' '*docket_number']


STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression [_logistic.py:814]


In [30]:
from sklearn.ensemble import AdaBoostClassifier

clf_2 = AdaBoostClassifier()
clf_2 = clf_2.fit(features_train, labels_train)
pred = clf_2.predict(features_test)
prob = clf_2.predict_proba(features_test)  # [:,1]
print("AdaBoost Forest")

print(accuracy_score(labels_test, pred))
# evaluate(prob,pred, labels_test)

print("")
print(labels_test, "\n")
print(pred)

AdaBoost Forest
0.3

['*users1_address_line_one' '*docket_number' '*trial_court_county'
 '*signature_date' '*users1_name' '*users1_signature' '*signature_date'
 '*users1_name' '*users1_birthdate' '*trial_court_county'] 

['_nonstandard_' '*docket_number' '*trial_court_county' '*docket_number'
 '*docket_number' '*users1_signature' '*docket_number' '*docket_number'
 '*docket_number' '*docket_number']


In [31]:
from sklearn.ensemble import GradientBoostingClassifier

clf_3 = GradientBoostingClassifier()
clf_3 = clf_3.fit(features_train, labels_train)
pred = clf_3.predict(features_test)
prob = clf_3.predict_proba(features_test)  # [:,1]
print("Gradient Boost")

print(accuracy_score(labels_test, pred))
# evaluate(prob,pred, labels_test)

print("")
print(labels_test, "\n")
print(pred)

Gradient Boost
1.0

['*users1_address_line_one' '*docket_number' '*trial_court_county'
 '*signature_date' '*users1_name' '*users1_signature' '*signature_date'
 '*users1_name' '*users1_birthdate' '*trial_court_county'] 

['*users1_address_line_one' '*docket_number' '*trial_court_county'
 '*signature_date' '*users1_name' '*users1_signature' '*signature_date'
 '*users1_name' '*users1_birthdate' '*trial_court_county']


In [32]:
from sklearn.ensemble import RandomForestClassifier

clf_4 = RandomForestClassifier()
clf_4 = clf_4.fit(features_train, labels_train)
pred = clf_4.predict(features_test)
prob = clf_4.predict_proba(features_test)  # [:,1]
print("Random Forest")

print(accuracy_score(labels_test, pred))
# evaluate(prob,pred, labels_test)

print("")
print(labels_test, "\n")
print(pred)

Random Forest
1.0

['*users1_address_line_one' '*docket_number' '*trial_court_county'
 '*signature_date' '*users1_name' '*users1_signature' '*signature_date'
 '*users1_name' '*users1_birthdate' '*trial_court_county'] 

['*users1_address_line_one' '*docket_number' '*trial_court_county'
 '*signature_date' '*users1_name' '*users1_signature' '*signature_date'
 '*users1_name' '*users1_birthdate' '*trial_court_county']


In [116]:
overwrite = input("Overwrite old data? (y/n)\n")
if "y" in overwrite.lower():
    dump(included_fields, "../formfyxer/data/included_fields.joblib")
    dump(clf_4, "../formfyxer/data/clf_field_names.joblib")
    dump(jurisdictions, "../formfyxer/data/jurisdictions.joblib")
    dump(groups, "../formfyxer/data/groups.joblib")

Overwrite old data? (y/n)
y
